# Flight Radar Analysis

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from geopy import distance 
from datetime import date, timedelta, datetime
import time
from FlightRadar24.api import FlightRadar24API

spark = SparkSession.builder.appName("FlightRadar")\
.config ("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

## - Load Airports

In [148]:
fr_api = FlightRadar24API()
rdd_1 = spark.sparkContext.parallelize(fr_api.get_airports())
schema_1 = StructType([StructField("name", StringType(), True),
                     StructField("iata", StringType(), True),
                     StructField("icao", StringType(), True),
                     StructField("lat", DoubleType(), True),
                     StructField("lon", DoubleType(), True),
                     StructField("country", StringType(), True),
                     StructField("alt", IntegerType(), True)])

airports = spark.createDataFrame(rdd_1, schema_1)
airports = airports.withColumnRenamed("icao","icao_airport")
airports = airports.drop("name","lat","lon",'alt') 
airports.show(10)

+----+------------+---------+
|iata|icao_airport|  country|
+----+------------+---------+
| KEU|        HKKE|    Kenya|
| LCG|        LECO|    Spain|
| AAH|        EDKA|  Germany|
| AAL|        EKYT|  Denmark|
| AAR|        EKAH|  Denmark|
| QEA|        EKAC|  Denmark|
| JEG|        BGAA|Greenland|
| ABD|        OIAA|     Iran|
| ABA|        UNAA|   Russia|
| YXX|        CYXX|   Canada|
+----+------------+---------+
only showing top 10 rows



## - Load Airlines

In [135]:
rdd_2 = spark.sparkContext.parallelize(fr_api.get_airlines())
schema_2 = StructType([StructField("Name", StringType(), True),
                     StructField("Code", StringType(), True),
                     StructField("ICAO", StringType(), True)])

airlines = spark.createDataFrame(rdd_2, schema_2)
airlines = airlines.withColumnRenamed("Name","airline_name")\
            .withColumnRenamed("Code","code_airline")\
            .withColumnRenamed("ICAO","icao_airline")
print((airlines.count(), len(airlines.columns)))
airlines.toPandas()

(1780, 3)


,airline_name,code_airline,icao_airline
0,21 Air,2I,CSB
1,25only Aviation,4Q,ONY
2,2Excel Aviation,,BRO
3,40-Mile Air,Q5,MLA
4,748 Air Services,FE,IHO
...,...,...,...
1775,Zil Air,,SYZ
1776,Zimex Aviation,XM,IMX
1777,Zimex Aviation Austria,,AZD
1778,ZIPAIR,ZG,TZP


## Compute flight informations

In [155]:
def compute_two_points(point1, point2):
    distance_2d= distance.distance(point1[:2], point2[:2]).m
    distance_3d = np.sqrt(distance_2d**2 + (point2[2] - point1[2])**2)
    return distance_3d

def compute_current_distance(positions):
    flight_distance = 0
    try:
        for i in range(len(positions) - 1):
            start = positions[i]
            end = positions[i + 1]
            start_info = [start['lat'], start['lng'], start['alt']]
            end_info = [end['lat'], end['lng'], end['alt']]
            if None in start_info or None in end_info:
                continue
            flight_distance += compute_two_points(start_info, end_info)
    except:
        return flight_distance
    return flight_distance
    
def get_value(details, path):
    value = details
    for key in path:
        try:
            value = value[key]
        except:
            return None
    return value

flights = fr_api.get_flights()
flight_infos = []

for i, flight in enumerate(flights):
    details = fr_api.get_flight_details(flight.id)
    #print(flight.id)

    icao_airline = get_value(details, ['airline','code','icao'])
    icao_airport = get_value(details, ['airport','origin','code','icao'])
    aircraft_model = get_value(details, ['aircraft','model','code'])
    aircraft_registration = get_value(details, ['aircraft','registration'])
    aircraft_manufacturer = get_value(details, ['aircraft','model','text'])
    aircraft_manufacturer = aircraft_manufacturer.split(' ')[0]  if not pd.isnull(aircraft_manufacturer) else None

    #origin infos
    airport_origin_name = get_value(details, ['airport','origin','name'])
    airport_origin_region = get_value(details, ['airport','origin','timezone','name'])
    airport_origin_region = airport_origin_region.split('/')[0] if not pd.isnull(airport_origin_region) else None
    airport_origin = get_value(details, ['airport','origin','position'])
    airport_origin_country = airport_origin['country']['name'] if not pd.isnull(airport_origin) else None
    airport_origin_lat = airport_origin.get('latitude') if not pd.isnull(airport_origin) else None
    airport_origin_lng = airport_origin.get('longitude') if not pd.isnull(airport_origin) else None
    airport_origin_alt = airport_origin.get('altitude') if not pd.isnull(airport_origin) else None

    #Destination infos
    airport_destination_name = get_value(details, ['airport','destination','name'])
    airport_destination_region = get_value(details, ['airport','destination','timezone','name'])
    airport_destination_region = airport_destination_region.split('/')[0] if not pd.isnull(airport_destination_region) else None
    airport_destination = get_value(details, ['airport','destination','position'])
    airport_destination_lat = airport_destination.get('latitude') if not pd.isnull(airport_destination) else None
    airport_destination_lng = airport_destination.get('longitude') if not pd.isnull(airport_destination) else None
    airport_destination_alt = airport_destination.get('altitude') if not pd.isnull(airport_destination) else None

    positions = get_value(details, ['trail'])
    try:
        curent_position = positions[-1]
    except:
        curent_position = None

    curent_lat = curent_position.get('lat') if not pd.isnull(curent_position) else None
    curent_lng = curent_position.get('lng') if not pd.isnull(curent_position) else None
    curent_alt = curent_position.get('alt') if not pd.isnull(curent_position) else None
    curent_spd = curent_position.get('spd') if not pd.isnull(curent_position) else None
    curent_ts = curent_position.get('ts') if not pd.isnull(curent_position) else None

    flight_distance = compute_current_distance(positions)

    flight_infos.append([icao_airline, icao_airport, aircraft_model, aircraft_registration, aircraft_manufacturer,
                         airport_origin_name, airport_origin_region, airport_origin_country, airport_origin_lat, airport_origin_lng, airport_origin_alt,
                         airport_destination_name, airport_destination_region, airport_destination_lat, airport_destination_lng, airport_destination_alt,
                         curent_lat, curent_lng, curent_alt, curent_spd, curent_ts, float(flight_distance)])




In [156]:
columns = ['icao_airline', 'icao_airport', 'aircraft_model', 'aircraft_registration', 'aircraft_manufacturer',
             'airport_origin_name', 'airport_origin_region', 'airport_origin_country', 'airport_origin_lat', 'airport_origin_lng', 'airport_origin_alt',
             'airport_destination_name', 'airport_destination_region', 'airport_destination_lat', 'airport_destination_lng', 'airport_destination_alt',
             'curent_lat', 'curent_lng', 'curent_alt', 'curent_spd', 'curent_ts', 'flight_distance']
rdd_4 = spark.sparkContext.parallelize(flight_infos)
flight_data = rdd_4.toDF(columns)
flight_data = flight_data.na.drop(subset=columns)

In [157]:
#Join flight with airline informations
all_flight_data = flight_data.join(airlines, flight_data.icao_airline ==  airlines.icao_airline,"inner")
all_flight_data.toPandas()

,icao_airline,icao_airport,aircraft_model,aircraft_registration,aircraft_manufacturer,airport_origin_name,airport_origin_region,airport_origin_country,airport_origin_lat,airport_origin_lng,...,airport_destination_alt,curent_lat,curent_lng,curent_alt,curent_spd,curent_ts,flight_distance,airline_name,code_airline,icao_airline
0,BOX,EDDF,B77L,D-AALI,Boeing,Frankfurt Airport,Europe,Germany,50.037796,8.555783,...,1026,50.036976,8.529696,0,0,1664096501,7.765364e+06,AeroLogic,3S,BOX
1,AFR,LFPG,A359,F-HTYQ,Airbus,Paris Charles de Gaulle Airport,Europe,France,49.012516,2.555752,...,645,48.999962,2.580185,0,17,1664115720,3.728656e+06,Air France,AF,AFR
2,AFR,LFPG,A320,F-HEPG,Airbus,Paris Charles de Gaulle Airport,Europe,France,49.012516,2.555752,...,2838,49.000740,2.596031,0,11,1664114705,2.152083e+06,Air France,AF,AFR
3,AFR,LFPG,A332,F-GZCE,Airbus,Paris Charles de Gaulle Airport,Europe,France,49.012516,2.555752,...,17,49.001804,2.554445,0,8,1664113981,3.921308e+06,Air France,AF,AFR
4,AFR,LFPG,B77W,F-GSQV,Boeing,Paris Charles de Gaulle Airport,Europe,France,49.012516,2.555752,...,21,49.000175,2.584407,0,11,1664110798,4.806721e+06,Air France,AF,AFR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,UAL,RJTT,B789,N29968,Boeing,Tokyo Haneda International Airport,Asia,Japan,35.552250,139.779602,...,672,35.545864,139.770233,0,2,1664095068,9.536647e+06,United Airlines,UA,UAL
905,UAL,RJAA,B772,N78005,Boeing,Tokyo Narita International Airport,Asia,Japan,35.764721,140.386307,...,96,35.761379,140.384262,0,0,1664091452,1.013881e+07,United Airlines,UA,UAL
906,UAL,KSFO,B789,N27965,Boeing,San Francisco International Airport,America,United States,37.618969,-122.374001,...,434,37.618835,-122.393906,0,0,1664088084,9.854395e+06,United Airlines,UA,UAL
907,UAL,KSFO,B77W,N2748U,Boeing,San Francisco International Airport,America,United States,37.618969,-122.374001,...,21,37.618187,-122.391174,0,0,1664085583,1.046923e+07,United Airlines,UA,UAL


## - Q1: What is the company with the most active flights in the world ?

In [159]:
all_flight_data.createOrReplaceTempView("all_flight_data")
spark.sql('''select airline_name, code_airline, count(*) as nb_active_airline from all_flight_data
            group by airline_name, code_airline order by nb_active_airline desc limit 1''').toPandas()

,airline_name,code_airline,nb_active_airline
0,Delta Air Lines,DL,59


## - Q2: By continent, what are the companies with the most regional active flights (airports of Origin & Destination within the same continent) ?

In [161]:
spark.sql('''with func as (select airline_name, code_airline, airport_destination_region as region, count(*) as nb_active_airline,
                max(count(*)) OVER (partition by airport_origin_region) AS hightest_airline from all_flight_data
                group by airline_name, code_airline, airport_origin_region, airport_destination_region 
                having airport_origin_region = airport_destination_region) 
            select airline_name, code_airline, region, hightest_airline from func 
            where nb_active_airline = hightest_airline
            order by hightest_airline desc
                ''').toPandas()

,airline_name,code_airline,region,hightest_airline
0,American Airlines,AA,America,34
1,Qatar Airways,QR,Asia,10
2,Jet2,LS,Europe,1
3,Air France,AF,Europe,1
4,Azur Air,ZF,Europe,1
5,Jet Story,,Europe,1
6,Red Wings,WZ,Europe,1
7,SunExpress,XQ,Europe,1
8,Corendon Airlines,XC,Europe,1
9,Aeroflot,SU,Europe,1


## - Q3: World-wide, Which active flight has the longest route ?

In [163]:
spark.sql('''select airline_name, code_airline, airport_origin_region, airport_destination_region, flight_distance from all_flight_data
                where flight_distance = (select max(flight_distance) from all_flight_data)''').toPandas()

,airline_name,code_airline,airport_origin_region,airport_destination_region,flight_distance
0,Singapore Airlines,SQ,America,Asia,1.502882e+07


## - Q4: By continent, what is the average route distance ? (flight localization by airport of origin)

In [164]:
spark.sql('''select airport_origin_region, mean(flight_distance) as average_distance from all_flight_data
                group by airport_origin_region''').toPandas()

,airport_origin_region,average_distance
0,Asia,6.574411e+06
1,America,4.626273e+06
2,Pacific,7.104151e+06
3,Atlantic,4.119535e+06
4,Europe,5.608437e+06
5,Australia,6.506782e+06
6,Indian,3.293693e+06
7,Africa,5.236969e+06


## - Q5.1: Which leading airplane manufacturer has the most active flights in the world ?

In [165]:
spark.sql('''select aircraft_manufacturer, count(*) as nb_active_airline from all_flight_data
            group by aircraft_manufacturer order by nb_active_airline desc limit 1''').toPandas()

,aircraft_manufacturer,nb_active_airline
0,Boeing,531


## - Q5.2: By continent, what is the most frequent airplane model ? (airplane localization by airport of origin)

In [168]:
spark.sql('''with func as (select aircraft_model, airport_origin_region as region, count(*) as nb_active_airline,
                max(count(*)) OVER (partition by airport_origin_region) AS hightest_airline from all_flight_data
                group by aircraft_model, airport_origin_region) 
            select region, aircraft_model, hightest_airline from func 
            where nb_active_airline = hightest_airline
            order by region, aircraft_model
                ''').toPandas()

,region,aircraft_model,hightest_airline
0,Africa,A333,2
1,Africa,B77W,2
2,Africa,B788,2
3,America,A321,30
4,Asia,B77W,37
5,Atlantic,B38M,2
6,Atlantic,B752,2
7,Australia,A388,4
8,Australia,B77W,4
9,Australia,B789,4


## - Q6: By company registration country, what are the tops 3 airplanes model flying ?

In [173]:
spark.sql('''select airport_origin_country, count(*) as nb_active_airline from all_flight_data
            group by airport_origin_country order by nb_active_airline desc limit 3''').toPandas()

,airport_origin_country,nb_active_airline
0,United States,207
1,United Kingdom,98
2,Germany,61


## - Q7.1: By continent, what airport is the most popular destination ?

In [194]:
spark.sql('''with func as (select airport_destination_name, airport_destination_region as continent, count(*) as nb_destination,
                max(count(*)) OVER (partition by airport_destination_region) AS hightest_destination from all_flight_data
                group by airport_destination_region, airport_destination_name) 
            select continent, airport_destination_name, hightest_destination from func 
            where nb_destination = hightest_destination
            order by hightest_destination desc limit 1
                ''').toPandas()

,continent,airport_destination_name,hightest_destination
0,America,Los Angeles International Airport,53


## - Q7.2: What airport airport has the greatest inbound/outbound flights difference ? (positive or negative)

In [193]:
spark.sql('''with taking_off as (select airport_origin_name,  count(*) as outbound from all_flight_data group by airport_origin_name),
                landing as (select airport_destination_name,  count(*) as inbound from all_flight_data group by airport_destination_name)
            select airport_origin_name as airport, outbound + inbound as outbound_inbound from taking_off to
            inner join landing ld on ld.airport_destination_name = to.airport_origin_name
            order by outbound_inbound desc limit 1
                ''').toPandas()

,airport,outbound_inbound
0,Los Angeles International Airport,81


## - Q8: By continent, what is the average active flight speed ? (flight localization by airport of origin)

In [196]:
spark.sql('''select airport_origin_region, mean(curent_spd) as average_speed from all_flight_data
            group by airport_origin_region''').toPandas()

,airport_origin_region,average_speed
0,Asia,25.802139
1,America,7.000000
2,Pacific,1.750000
3,Atlantic,25.142857
4,Europe,5.235154
5,Australia,0.473684
6,Indian,0.000000
7,Africa,82.090909
